# Model Fit and Image Comparison Pipeline

The purpose of this notebook is to load a series of images fit blindly to data and compare the following metrics:

* Ring properties, such as

    * Radius
    * Width
    * Asymmetry
    * Intensity profiles along the ring
    * Flux inside the ring vs flux outside the ring
    * ...and more

Geometric properties, such as the radius, will be fit using a Hough transfrom and the `HoughTransform` library. Asymmetry, ring width, and flux inside and outside the ring will be determined using the `PontiFEX` library. 

These attributes are ring-specific. 


### 1. Load images.
Please provide your images and model fits as a .fits file, named as [naming convention]. For those submitting multiple images or model fits, please [multiple submission naming convention].

Note: proposed naming convention, will be used for now: `LastnameFirstname_YYMMDD_ID.fits`, where

 * `Lastname`, `Firstname` (self explanatory) ex: JohnsonMichael
 * `YYMMDD`, date of SUBMISSION (e.g. November 8th, 2018 becomes 181108)
 * `ID`, dataset ID unique to each challenge. For the first challenge, this tag is: `WCROMH`. See: https://docs.google.com/spreadsheets/d/11oCD7T6okr3iRfJjvlFesowY3O8QLhv87HbOpP79v3c/edit?usp=sharing
 
 Example submission file name: `JohnsonMichael_181108_WCROMH.fits`

In [23]:
## imports ##
import glob
import Pontifex
import itertools
import ehtim as eh
import seaborn as sns
import HoughTransform

## graph settings ##
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.linewidth'] = 2 #set the value globally
plt.rcParams["font.weight"] = "bold"
plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
cycol = itertools.cycle(sns.color_palette())

## ignore warnings ##
import warnings
warnings.filterwarnings('ignore')


## settings ##
RUN_ALL = True

In [27]:
## quick class to store image submissions and store the results of their analysis ##
class Submission(object):
    
    def __init__(self, _filepath, run_all_analyses=False):
        ## pre-analysis attributes ##
        self._filepath      = _filepath
        self.submitter      = self._filepath.split['/'][-1].split('_')[0]
        self.submissionDate = int(self._filepath.split['/'][-1].split('_')[1])
        self.challengeID    = self._filepath.split['/'][-1].split('_')[-1].split('.')[0]
        self.image          = self.loadImage(self._filepath)
        self.ImageAnalysisObject = Pontifex.AnalysisObject(self.image, intervals=15)
        
        if run_all_analyses is True: self.runAll()
            
        
        ## post-analysis attributes ##
        self.analysisResults = {
            'radius':-1,
            'center':(0, 0),
            'width':-1,
            'asymmetry':-1,
            'fluxInside':-1,
            'fluxOutside':-1
        }
        
        ## intensity profiles in the cardinal directions ##
        self.intensityProfiles = {
            0:[],
            1:[],
            2:[],
            3:[]
        }
        
        
    def loadImage(self_fp):
        return eh.image.load_image(_fp)

    def runAll(self):
        ## run all the analyses ##
        self.getRadius()
        self.getAsymmetry()
        self.getWidth()
        self.fluxInside()
        self.fluxOutside()
    
    def getRadius(self):
        ## use PontiFEX to get the mean radius ##
        while 1:
            try:
                self.ImageAnalysisObject.iterative_peak_finding()
                break
            except ValueError:
                self.ImageAnalysisObject.iterative_peak_finding()
        self.analysisResults['radius'] = ImageAnalysisObject.radius_dictionary['combined']
        
    def getAsymmetry(self):
        ## use PontiFEX to get the asymmetry ##
        print 'asymmetry'
    
    def getWidth(self):
        ## use PontiFEX to get the mean width ##
        print 'width'
    
    def fluxInside(self):
        ## use PontiFEX to get the flux inside ##
        print 'flux inside'
    
    def fluxOutside(self):
        ## use PontiFEX to get the flux outside ##
        print 'flux outside'
        

In [28]:
## load images from folder ##
_imagesFilepath = './sample_images/'
imageFiles = glob.glob(_imagesFilepath + '*.fits')

## create a submission object for each image ##
submissionsList = [Submission(_file, run_all_analyses=RUN_ALL) for _file in imageFiles]

TypeError: 'builtin_function_or_method' object has no attribute '__getitem__'